# Đọc input đầu vào

In [ ]:
import pandas as pd

# Đọc dữ liệu từ file Excel
# file_path = "C://Users//Computer//AppData//Local//Temp//Rar$DIa0.358//h100r202.csv"  # Thay bằng đường dẫn file thực tế
file_path = "C://Users//hungdm129//Documents//Zalo Received Files//Data_VRP_Drone//h100r202.csv"  # Thay bằng đường dẫn file thực tế
df = pd.read_csv(file_path)

# Lưu từng cột vào danh sách riêng biệt
x_coords = df["x"].tolist()  # Danh sách tọa độ x
y_coords = df["y"].tolist()  # Danh sách tọa độ y
demands = df["demand"].tolist()  # Nhu cầu hàng hóa
eet = df["eet"].tolist()  # Earliest End Time
open_time = df["open"].tolist()  # Giờ mở cửa
close_time = df["close"].tolist()  # Giờ đóng cửa
elt = df["elt"].tolist()  # Earliest Latest Time
service_time = df["servicetime"].tolist()  # Thời gian phục vụ
drone_serve = df["drone_serve"].tolist()  # Cờ có thể dùng drone

time_windows = [list(item) for item in zip(eet, open_time, close_time, elt)]




             x          y  demand  eet  open  close   elt  servicetime  \
0    29.166667  29.166667       0    0     0   1000  1020            5   
1    34.166667  40.833333      10    0     0    974   983            5   
2    29.166667  14.166667       7    0     0    972   993            5   
3    45.833333  37.500000      13    0     0    967   984            5   
4    12.500000  25.000000      26    0     0    969   985            5   
..         ...        ...     ...  ...   ...    ...   ...          ...   
96   47.500000  24.166667      18  624   641    734   751            5   
97   37.500000  54.166667       9  559   581    666   688            5   
98   44.166667  10.000000       6  574   585    692   703            5   
99   21.666667  29.166667      15  748   755    920   927            5   
100  30.833333  46.666667       5  764   788    968   992            5   

     drone_serve  
0              1  
1              0  
2              1  
3              0  
4              0

# Tính toán ma trận khoảng cách

In [2]:
from math import sqrt
import numpy as np

# Số lượng điểm (depot + khách hàng)
n_points = len(x_coords)

# Tính ma trận khoảng cách
def calculate_distance_matrix(x_coords, y_coords):
    dist_matrix = np.zeros((n_points, n_points))
    for i in range(n_points):
        for j in range(n_points):
            dist_matrix[i][j] = sqrt((x_coords[i] - x_coords[j])**2 + (y_coords[i] - y_coords[j])**2)
    return dist_matrix

dist_matrix = calculate_distance_matrix(x_coords, y_coords)

print(dist_matrix)

[[ 0.         12.69295517 15.         ... 24.33846978  7.5
  17.57918592]
 [12.69295517  0.         27.13136765 ... 32.41441723 17.09857044
   6.71854813]
 [15.         27.13136765  0.         ... 15.56795141 16.77050983
  32.54270698]
 ...
 [24.33846978 32.41441723 15.56795141 ...  0.         29.55691309
  39.01566637]
 [ 7.5        17.09857044 16.77050983 ... 29.55691309  0.
  19.75544932]
 [17.57918592  6.71854813 32.54270698 ... 39.01566637 19.75544932
   0.        ]]


# Khởi tạo NST

In [3]:
from chromosome_utils import initialize_chromosome, Chromosome

# Thông số
n_trucks = 20  # Số xe tải (mỗi xe có 1 drone)

if n_trucks > n_points:
    raise Exception("Số lượng xe tải không được lớn hơn số lượng điểm")

chromosome = initialize_chromosome(n_points, n_trucks, drone_serve)
print("Tuyến xe tải:", chromosome.truck_routes)
print("Phân công drone:", chromosome.drone_assignment)

Tuyến xe tải: [0, 15, 19, 30, 35, 14, 0, 46, 95, 62, 50, 20, 0, 73, 3, 66, 98, 91, 0, 44, 97, 7, 17, 25, 0, 32, 60, 40, 79, 68, 0, 78, 57, 12, 55, 29, 0, 41, 74, 43, 90, 77, 0, 80, 70, 69, 45, 22, 0, 51, 27, 76, 100, 5, 0, 85, 49, 71, 82, 93, 0, 28, 1, 47, 9, 37, 0, 67, 99, 4, 81, 31, 0, 13, 16, 38, 96, 8, 0, 92, 42, 59, 24, 21, 0, 75, 83, 10, 58, 6, 0, 39, 88, 65, 94, 64, 0, 23, 72, 34, 56, 87, 0, 48, 54, 36, 86, 89, 0, 2, 26, 84, 53, 11, 0, 52, 63, 61, 33, 18, 0]
Phân công drone: [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]


# validate NST

In [4]:
from validation import is_valid_chromosome, ERROR_TYPES

chromosome.truck_routes = [0, 7, 51, 76, 81, 74, 34, 69, 14, 21, 11, 35, 52, 90, 57, 19, 47, 60, 55, 25, 87, 63, 0, 62, 33, 80, 5, 46, 48, 82, 83, 61, 45, 0, 4, 93, 58, 40, 28, 15, 22, 53, 8, 97, 0, 64, 67, 9, 84, 77, 91, 3, 32, 23, 86, 0, 50, 70, 71, 56, 96, 95, 17, 85, 73, 13, 0, 12, 20, 72, 49, 59, 89, 79, 2, 78, 31, 0, 94, 43, 27, 24, 65, 18, 75, 26, 10, 6, 0, 99, 39, 37,  100, 66, 30, 41, 42, 16, 0, 98, 44, 1, 54, 38, 68, 36, 88,0, 92,29, 0]

if is_valid_chromosome(chromosome, n_points, n_trucks, demands, Q=1000):
    print("NST hợp lệ!")

# start = 0
# routes = []
# for i in range(1, len(chromosome.truck_routes)):
#         if chromosome.truck_routes[i] == 0:
#             routes.append(chromosome.truck_routes[start:i + 1])
#             start = i
# if not routes:
#     routes.append(chromosome.truck_routes)
# print(routes)
# for route in routes:
#     print(route[len(route)-2])
#     print(route)

Lỗi (5)/(6): Có xe tải không được sử dụng hoặc sử dụng nhiều lần (Số 0 (11) không khớp với n_trucks + 1 (21))
NST hợp lệ!


# tính toán các đối tượng

In [5]:
from evaluate import evaluate_objectives

makespan, truck_distance, drone_distance, avg_satisfaction = evaluate_objectives(
    chromosome, x_coords, y_coords, 40, 80, service_time, n_trucks, time_windows
)

print("Makespan:", makespan)
print("Truck distance:", truck_distance)
print("Drone distance:", drone_distance)
print("Average satisfaction:", avg_satisfaction)


True
Makespan: 884.9113600162942
Truck distance: 2171.3907232050497
Drone distance: 1482.949796530549
Average satisfaction: 0.3376111000020717


# local search repair


In [6]:
from local_search_repair import local_search_repair

valid, error_type = is_valid_chromosome(chromosome, n_points, n_trucks, demands, 1000)
if not valid:
    print(f"Chromosome không hợp lệ: {ERROR_TYPES[error_type]}. Đang sửa bằng local search...")
    chromosome = local_search_repair(chromosome, n_points, n_trucks, demands, 1000)
    print("Chromosome sau khi sửa:", chromosome.truck_routes)
    valid, error_type = is_valid_chromosome(chromosome, n_points, n_trucks, demands, 1000)
    if valid:
        print("Chromosome đã được sửa thành hợp lệ!")
        print(chromosome.truck_routes)
    else:
        print(f"Chromosome không hợp lệ: {ERROR_TYPES[error_type]}")
else:
    print("Chromosome hợp lệ!")

Lỗi (5)/(6): Có xe tải không được sử dụng hoặc sử dụng nhiều lần (Số 0 (11) không khớp với n_trucks + 1 (21))
Chromosome không hợp lệ: WRONG_TRUCK_COUNT. Đang sửa bằng local search...
Lỗi (5)/(6): Có xe tải không được sử dụng hoặc sử dụng nhiều lần (Số 0 (11) không khớp với n_trucks + 1 (21))
Đang sửa lỗi: WRONG_TRUCK_COUNT
Chromosome sau khi sửa: [0, 7, 51, 76, 81, 74, 0, 34, 69, 14, 21, 11, 0, 35, 52, 90, 57, 19, 0, 47, 60, 55, 25, 87, 0, 63, 62, 33, 80, 5, 0, 46, 48, 82, 83, 61, 0, 45, 4, 93, 58, 40, 0, 28, 15, 22, 53, 8, 0, 97, 64, 67, 9, 84, 0, 77, 91, 3, 32, 23, 0, 86, 50, 70, 71, 56, 0, 96, 95, 17, 85, 73, 0, 13, 12, 20, 72, 49, 0, 59, 89, 79, 2, 78, 0, 31, 94, 43, 27, 24, 0, 65, 18, 75, 26, 10, 0, 6, 99, 39, 37, 100, 0, 66, 30, 41, 42, 16, 0, 98, 44, 1, 54, 38, 0, 68, 36, 88, 92, 29, 0]
Chromosome đã được sửa thành hợp lệ!
[0, 7, 51, 76, 81, 74, 0, 34, 69, 14, 21, 11, 0, 35, 52, 90, 57, 19, 0, 47, 60, 55, 25, 87, 0, 63, 62, 33, 80, 5, 0, 46, 48, 82, 83, 61, 

# MOEA/D

In [7]:
# Order Crossover (OX)
def order_crossover(parent1, parent2):
    customers1 = [x for x in parent1.truck_routes if x != 0]
    customers2 = [x for x in parent2.truck_routes if x != 0]
    size = len(customers1)
    start, end = sorted([random.randint(0, size - 1) for _ in range(2)])
    
    child_customers = [-1] * size
    for i in range(start, end + 1):
        child_customers[i] = customers1[i]
    
    remaining = [x for x in customers2 if x not in child_customers]
    j = 0
    for i in range(size):
        if child_customers[i] == -1:
            child_customers[i] = remaining[j]
            j += 1
    
    truck_routes = [0]
    n_trucks = parent1.truck_routes.count(0) - 1
    customers_per_truck = size // n_trucks
    for i in range(n_trucks - 1):
        truck_routes.extend(child_customers[i * customers_per_truck:(i + 1) * customers_per_truck])
        truck_routes.append(0)
    truck_routes.extend(child_customers[(n_trucks - 1) * customers_per_truck:])
    truck_routes.append(0)
    
    drone_assignment = [0] * len(parent1.drone_assignment)
    for i in range(len(drone_assignment)):
        drone_assignment[i] = random.choice([parent1.drone_assignment[i], parent2.drone_assignment[i]])
    
    return Chromosome(truck_routes, drone_assignment)

# Swap Mutation
def swap_mutation(chromosome):
    customers = [x for x in chromosome.truck_routes if x != 0]
    if len(customers) < 2:
        return chromosome
    i, j = random.sample(range(len(customers)), 2)
    customers[i], customers[j] = customers[j], customers[i]
    
    truck_routes = [0]
    n_trucks = chromosome.truck_routes.count(0) - 1
    customers_per_truck = len(customers) // n_trucks
    for k in range(n_trucks - 1):
        truck_routes.extend(customers[k * customers_per_truck:(k + 1) * customers_per_truck])
        truck_routes.append(0)
    truck_routes.extend(customers[(n_trucks - 1) * customers_per_truck:])
    truck_routes.append(0)
    
    drone_assignment = chromosome.drone_assignment.copy()
    if random.random() < 0.1:  # Xác suất đột biến drone_assignment
        idx = random.randint(1, len(drone_assignment) - 1)
        drone_assignment[idx] = 1 - drone_assignment[idx]
    
    return Chromosome(truck_routes, drone_assignment)

# Tchebycheff Aggregation
def tchebycheff(chromosome, weight, z, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q):
    makespan, truck_dist, drone_dist, avg_sat = evaluate_objectives(chromosome, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q)
    # Chuyển mức độ hài lòng thành tối thiểu hóa (1 - avg_sat)
    objectives = [makespan, truck_dist, drone_dist, 1 - avg_sat]
    return max(weight[i] * abs(objectives[i] - z[i]) for i in range(4))

# MOEA/D
def moead(x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q=1000, pop_size=100, generations=50):
    n_objectives = 4
    population = [initialize_chromosome(n_points, n_trucks, drone_serve) for _ in range(pop_size)]
    
    # Khởi tạo trọng số
    weights = []
    for i in range(pop_size):
        w = np.random.dirichlet(np.ones(n_objectives), size=1)[0]
        weights.append(w)
    
    # Tìm láng giềng cho mỗi bài toán con
    B = []
    for i in range(pop_size):
        distances = [np.linalg.norm(weights[i] - weights[j]) for j in range(pop_size)]
        B.append(np.argsort(distances)[:10].tolist())  # 10 láng giềng gần nhất
    
    # Giá trị lý tưởng (z)
    z = [float('inf')] * n_objectives  # Tối thiểu hóa
    for ind in population:
        makespan, truck_dist, drone_dist, avg_sat = evaluate_objectives(ind, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q)
        z[0] = min(z[0], makespan)
        z[1] = min(z[1], truck_dist)
        z[2] = min(z[2], drone_dist)
        z[3] = min(z[3], 1 - avg_sat)  # Tối thiểu hóa 1 - avg_sat
    
    # Vòng lặp chính
    for gen in range(generations):
        for i in range(pop_size):
            # Chọn ngẫu nhiên 2 láng giềng
            neighbors = random.sample(B[i], 2)
            parent1, parent2 = population[neighbors[0]], population[neighbors[1]]
            
            # Tạo cá thể mới
            child = order_crossover(parent1, parent2)
            child = swap_mutation(child)
            
            # Đánh giá và cập nhật z
            makespan, truck_dist, drone_dist, avg_sat = evaluate_objectives(child, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q)
            z[0] = min(z[0], makespan)
            z[1] = min(z[1], truck_dist)
            z[2] = min(z[2], drone_dist)
            z[3] = min(z[3], 1 - avg_sat)
            
            # Cập nhật láng giềng
            for j in B[i]:
                if tchebycheff(child, weights[j], z, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q) < \
                   tchebycheff(population[j], weights[j], z, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q):
                    population[j] = child
        
        print(f"Thế hệ {gen + 1}/{generations} hoàn tất.")
    
    # Trả về tập Pareto
    pareto_front = []
    for ind in population:
        objectives = evaluate_objectives(ind, x_coords, y_coords, v_truck, v_drone, service_time, n_trucks, time_windows, demands, Q)
        pareto_front.append((ind, objectives))
    
    return pareto_front

# execute

In [8]:
pareto_front = moead(x_coords, y_coords, 40, 80, service_time, n_trucks, time_windows, demands, 1000, pop_size=50, generations=20)

TypeError: evaluate_objectives() takes 8 positional arguments but 10 were given